In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import re
import os
import matplotlib.pyplot as plt
import nltk
from collections import OrderedDict

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to /home/riya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/riya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/riya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
relative_path_sdg = "../../sdg_data.csv"
relative_path_attr = "../Dictionary/Attributes_2016-17.csv"
relative_string_map = "./Data_Target/String_map.csv"
relative_semantic = "./Data_Target/Semantic_map.csv"
relative_sem_threshold = "./Data_Target/Semantic_threshold_map.csv"
relative_rank = "./Data_Target/Ranking.csv"
relative_rank_unroll = "./Data_Target/Ranking_unroll.csv"
relative_tab_filter = "./Data_Target/Ranking_tab_filter.csv"
relative_chap_filter = "./Data_Target/Ranking_chap_filter.csv"
relative_final_map = "./Mapping_Target_2016-17.csv"
relative_count = "./Data_Target/Comparison_count.csv"


# Get the absolute path by joining the current directory with the relative path
absolute_path_sdg = os.path.normpath(os.path.join(os.getcwd(), relative_path_sdg))
absolute_path_attr = os.path.normpath(os.path.join(os.getcwd(), relative_path_attr))
absolute_path_string = os.path.normpath(os.path.join(os.getcwd(), relative_string_map))
absolute_path_semantic = os.path.normpath(os.path.join(os.getcwd(), relative_semantic))
absolute_path_sem_threshold =os.path.normpath( os.path.join(os.getcwd(), relative_sem_threshold))
absolute_path_rank =os.path.normpath( os.path.join(os.getcwd(), relative_rank))
absolute_path_rank_unroll =os.path.normpath( os.path.join(os.getcwd(), relative_rank_unroll))
absolute_path_tab_filter =os.path.normpath( os.path.join(os.getcwd(), relative_tab_filter))
absolute_path_chap_filter =os.path.normpath( os.path.join(os.getcwd(), relative_chap_filter))
absolute_path_final_map =os.path.normpath( os.path.join(os.getcwd(), relative_final_map))
absolute_path_count =os.path.normpath( os.path.join(os.getcwd(), relative_count))

In [3]:
def process_result(result_sdg, sdg_df, chapter_df):
    rows = []
    
    for _, row in result_sdg.iterrows():
        attr_id = row['Rank']
        goal_no = row['Goal No.']
        target_id = row['Target_id']
        
        for attr in attr_id:
            rows.append({
                'Attr_id': attr,
                'top_n_goal_id': goal_no,
                'top_n_target_id': target_id
            })

    all_goals = pd.DataFrame(rows)
    
    goal_name = dict(zip(sdg_df['Goal No.'], sdg_df['Goal']))
    target_name = dict(zip(sdg_df['Target_id'], sdg_df['Targets']))
    
    all_goals['Goal'] = all_goals['top_n_goal_id'].map(goal_name.get)
    all_goals['Targets'] = all_goals['top_n_target_id'].map(target_name.get)
    chapter_id = dict(zip(chapter_df['Attr_id'], chapter_df['Chapter_id']))
    chapter_name = dict(zip(chapter_df['Chapter_id'], chapter_df['Chapter_name']))
    table_id = dict(zip(chapter_df['Attr_id'], chapter_df['Table_id']))
    table_name = dict(zip(chapter_df['Table_id'], chapter_df['Table_name']))
    description = dict(zip(chapter_df['Attr_id'], chapter_df['Description']))
    all_goals['Chapter_id'] = all_goals['Attr_id'].map(chapter_id.get)
    all_goals['Chapter_name'] = all_goals['Chapter_id'].map(chapter_name.get)
    all_goals['Table_id'] = all_goals['Attr_id'].map(table_id.get)
    all_goals['Table_name'] = all_goals['Table_id'].map(table_name.get)
    all_goals['Description'] = all_goals['Attr_id'].map(description.get) 
    
                
    all_goals = all_goals.loc[:,['Attr_id','Chapter_id', 'Chapter_name', 'Table_id', 'Table_name', 'Description', 'Goal', 'Targets', 'top_n_goal_id', 'top_n_target_id']]

    return all_goals


In [4]:
# Function to combine and remove duplicates from lists while maintaining order
def combine_lists_BC(row):
    list_B = row['top_n_attr_B']
    list_C = row['top_n_attr_C']
    
    # Combine lists and remove duplicates while maintaining order
    items = list(OrderedDict.fromkeys(list_B + list_C))
    return items

In [5]:
# Function to find intersection while maintaining order
def find_intersection_ABC(row):
    intersection = [x for x in row['top_n_attr_A'] if x in row['B_plus_C']]
    return intersection

In [6]:
def find_difference_A(row):
    top_n_attr = row['top_n_attr_A']
    intersection = row['A_and_BC']
    
    # Remove elements in BC and intersection from top_n_attr while preserving order
    difference = [x for x in top_n_attr if x not in intersection]
    
    return difference

In [7]:
# Function to find intersection while maintaining order
def find_intersection_BC(row):
    intersection = [x for x in row['top_n_attr_B'] if x in row['top_n_attr_C']]
    return intersection

In [8]:
def find_difference_B(row):
    top_n_attr = row['top_n_attr_B']
    intersection = set(row['A'] + row['B_and_C'] + row['A_and_BC'])
    
    # Remove elements in BC and intersection from top_n_attr while preserving order
    difference = [x for x in top_n_attr if x not in intersection]
    
    return difference

In [9]:
def find_difference_C(row):
    top_n_attr = row['top_n_attr_C']
    intersection = set(row['A'] + row['B_and_C'] + row['A_and_BC'] + row['B_minus_all'])
    
    # Remove elements in BC and intersection from top_n_attr while preserving order
    difference = [x for x in top_n_attr if x not in intersection]
    
    return difference

In [10]:
def final_rank(row):
    #combined_top_n_attr = sorted(set(list_B + list_C), key=lambda x: (list_B + list_C).index(x))
    items = row['A_and_BC'] + row['A'] + row['B_and_C'] + row['B_minus_all'] + row['C_minus_all']
    item_set = list(OrderedDict.fromkeys(items))    
    return item_set

In [11]:
sdg = pd.read_csv(absolute_path_sdg, sep=",", dtype={'Goal No.': str, 'Target_id': str, 'Indicator_id':str})

In [12]:
sdg

,Goal No.,Goal,Nodal Department,Targets,Other Related Major Departments,Tentative Indicators,Target_id,Indicator_id
0,1,End poverty in all its forms everywhere,Rural Development,"By 2030, eradicate extreme poverty for all peo...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population below the interna...,1.1,1.1.1
1,1,End poverty in all its forms everywhere,Rural Development,"By 2030, reduce at least by half the proportio...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population living below the ...,1.2,1.2.1
2,1,End poverty in all its forms everywhere,Rural Development,"By 2030, reduce at least by half the proportio...","Urban Development, Agriculture, Horticulture, ...","Proportion of men, women and children of all a...",1.2,1.2.2
3,1,End poverty in all its forms everywhere,Rural Development,Implement nationally appropriate social protec...,"Urban Development, Agriculture, Horticulture, ...",Percentage of the population covered by social...,1.3,1.3.1
4,1,End poverty in all its forms everywhere,Rural Development,"By 2030, ensure that all men and women, in par...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population living in househo...,1.4,1.4.1
...,...,...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,Finance,"Promote the development, transfer, disseminati...",Environment and Scientific Technology.,Total amount of approved funding to promote th...,16.4,16.4.1
175,16,Strengthen the means of implementation and rev...,Finance,Fully operationalize the technology bank and s...,"Environment and Scientific Technology, Inform...",Proportion of individuals using the Internet.,16.5,16.5.1
176,17,"Data, monitoring and accountability",Finance,"By 2020, enhance capacity-building support to ...","Planning, Finance, Economic and Statistics.",Proportion of sustainable development indicato...,17.1,17.1.1
177,17,"Data, monitoring and accountability",Finance,"By 2030, build on existing initiatives to deve...","Planning, Finance, Economic and Statistics.",Dollar value of all resources made available t...,17.2,17.2.1


In [13]:
sdg.drop(['Nodal Department','Other Related Major Departments', 'Tentative Indicators', 'Indicator_id'],axis=1,inplace=True)

In [14]:
# Remove duplicate rows
sdg = sdg.drop_duplicates()

In [15]:
sdg.reset_index(inplace = True, drop = True)

In [16]:
data = pd.read_csv(absolute_path_attr, sep=";", dtype={'Attr_id': str, 'Chapter_id': str, 'Table_id':str})

In [17]:
data_new = data.copy()
data_new

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description
0,3.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,Nada Offices
1,4.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,Va Circles
2,5.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,Hoblies
3,6.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,Grama Panchayaths
4,7.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,No.of Taluks
...,...,...,...,...,...,...
1162,1169.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Total,Number,Total Agrl. Land Holder Total"
1163,1170.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Male,Area,Total Agrl. Land Holder Total"
1164,1171.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Female,Area,Total Agrl. Land Holder Total"
1165,1172.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Institutions,Area,Total Agrl. Land Holder Total"


In [18]:
sdg_string = pd.read_csv(absolute_path_string, sep=";", dtype={'Goal No.': str, 'Target_id': str, 'Indicator_id':str}, converters={'top_n_attr': pd.eval, 'top_n_table_id': pd.eval, 'top_n_chapter_id': pd.eval,'top_n_similarities': pd.eval})
sdg_string

,Goal No.,Goal,Targets,Target_id,new_targets,new_targets_str,top_n_count,top_n_attr,top_n_table_id,top_n_table_name,top_n_chapter_id,top_n_chapter_name,top_n_similarities,top_n_description
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1.1,"['eradicate', 'extreme', 'poverty', 'people', ...",by eradicate extreme poverty for all people ev...,30,"[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35....","[1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 1.7, 1.7, 1.7, ...",['Ration Shops And below poverty line Ration C...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","['General Information', 'General Information',...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","['Urban,Ration shops, Ration Shops And below p..."
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",1.2,"['reduce', 'least', 'half', 'men', 'woman', 'c...",by reduce at least by half the proportion of m...,94,"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[10.8, 10.8, 10.8, 10.8, 10.8, 10.8, 13.1, 13....",['No .of Children in the age group 6 to 14 yea...,"[10, 10, 10, 10, 10, 10, 13, 13, 13, 13, 13, 1...","['Education', 'Education', 'Education', 'Educa...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","['Boys,No.of Children Enrolment age group betw..."
2,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,1.3,"['implement', 'nationally', 'appropriate', 'so...",implement nationally appropriate social protec...,156,"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[11.3, 11.3, 11.3, 11.3, 11.3, 11.3, 11.3, 11....",['Allopathy Hospitals Indian System No.of Medi...,"[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 4, 10...","['Health & Family welfare Services', 'Health &...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, ...","['Nos.,Allopathy Hospitals, Allopathy Hospital..."
3,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",1.4,"['ensure', 'men', 'woman', 'particular', 'poor...",by ensure that all men and women in particular...,336,"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[11.3, 11.3, 11.3, 11.3, 11.3, 11.3, 11.3, 11....",['Allopathy Hospitals Indian System No.of Medi...,"[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 1...","['Health & Family welfare Services', 'Health &...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","['Nos.,Allopathy Hospitals, Allopathy Hospital..."
4,1,End poverty in all its forms everywhere,"By 2030, build the resilience of the poor and ...",1.5,"['build', 'resilience', 'poor', 'vulnerable', ...",by build the resilience of the poor and those ...,12,"[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[17.3, 17.3, 17.3, 17.3, 17.3, 17.3, 17.3, 17....","['Registration of Birth and Death', 'Registrat...","[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17]","['Miscellaneous', 'Miscellaneous', 'Miscellane...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","['Birth,Registered events Year 2016, Registrat..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,16,Strengthen the means of implementation and rev...,"Enhance cooperation on and access to science, ...",16.3,"['enhance', 'cooperation', 'access', 'science'...",enhance cooperation on and access to science t...,18,"[206.0, 523.0, 524.0, 525.0, 526.0, 527.0, 528...","[4.3, 9.3, 9.3, 9.3, 9.3, 9.3, 9.3, 11.3, 11.3...","['Area under principal crops Year 2015-16', 'R...","[4, 9, 9, 9, 9, 9, 9, 11, 11, 11, 11, 11, 11, ...","['Agriculture, Horticulture and Sericulture', ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","['Total Pulses including Tur Bengalgram,Pulses..."
127,16,Strengthen the means of implementation and rev...,"Promote the development, transfer, disseminati...",16.4,"['promote', 'development', 'transfer', 'dissem...",promote the development transfer dissemination...,56,"[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...","[13.1, 13.1, 13.1, 13.1, 13.1, 13.2, 13.2, 13....",['Stree Shakti Self He

In [19]:
sdg_sim = pd.read_csv(absolute_path_semantic, sep=";", dtype={'Goal No.': str, 'Target_id': str, 'Indicator_id':str}, converters={'top_n_attr': pd.eval, 'top_n_table_id': pd.eval, 'top_n_chapter_id': pd.eval,'top_n_similarities': pd.eval})
sdg_sim

,Goal No.,Goal,Targets,Target_id,new_targets,new_targets_str,keywords,vec,top_n_index,min_values,top_n_count,top_n_attr,top_n_chapter_id,top_n_table_id,top_n_description,top_n_chapter_name,top_n_table_name
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1.1,"['eradicate', 'extreme', 'poverty', 'people', ...",by eradicate extreme poverty for all people ev...,"[['poverty', 'people'], ['measured', 'people']...",[[-0.27236 0.229701 0.238934 ... 0.3...,"[132, 133, 376, 710, 711, 869, 870, 871, 875, ...","[0.8575334699106769, 0.5624979888636186, 0.629...",54,"[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[3, 3, 5, 11, 11, 14, 14, 14, 14, 3, 1, 1, 1, ...","[3.1, 3.1, 5.4, 11.6, 11.6, 14.3, 14.3, 14.3, ...","['Working,Raingauge Stations, No.of Rainguage ...","['Rainfall', 'Rainfall', 'Animal Husbandry', '...",['No.of Rainguage Raingauge Stations Annual No...
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",1.2,"['reduce', 'least', 'half', 'men', 'woman', 'c...",by reduce at least by half the proportion of m...,"[['reduce', 'least'], ['according', 'national'...",[[-0.070615 0.367065 0.00489179 ... -0.2...,"[594, 869, 511, 872, 873, 874, 875, 956, 833, ...","[0.6002113658853019, 0.5844719405576027, 0.574...",60,"[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[10, 14, 9, 14, 14, 14, 14, 17, 13, 13, 13, 13...","[10.11, 14.3, 9.2, 14.3, 14.3, 14.3, 14.3, 17....","['Men,No. of Lecturers, No.of P U Colleges Stu...","['Education', 'Rural Development & Panchayat R...",['No.of P U Colleges Students and Lecturers Ye...
2,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,1.3,"['implement', 'nationally', 'appropriate', 'so...",implement nationally appropriate social protec...,"[['appropriate', 'social'], ['protection', 'sy...",[[-0.60930499 -0.269645 -0.01956 ... 0.1...,"[198, 659, 660, 661, 662, 663, 664, 665, 666, ...","[0.8510072365400712, 0.7242761885702063, 1.000...",55,"[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[4, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12...","[4.3, 11.3, 11.3, 11.3, 11.3, 11.3, 11.3, 11.3...","['Total Pulses including Tur Bengalgram,Pulses...","['Agriculture, Horticulture and Sericulture', ...","['Area under principal crops Year 2015-16', 'A..."
3,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",1.4,"['ensure', 'men', 'woman', 'particular', 'poor...",by ensure that all men and women in particular...,"[['well'], ['particular', 'poor'], ['financial...",[[-0.13508999 0.35907 0.1453 ... -0.4...,"[169, 170, 171, 172, 173, 174, 882, 883, 903, ...","[0.7045711183911056, 0.5746299980541799, 0.678...",33,"[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[4, 4, 4, 4, 4, 4, 14, 14, 15, 13, 13, 13, 13,...","[4.2, 4.2, 4.2, 4.2, 4.2, 4.2, 14.5, 14.5, 15....","['Nos,Wells,2015-16 Net area irrigated Hectare...","['Agriculture, Horticulture and Sericulture', ...",['Gross and Net area Irrigated under Different...
4,1,End poverty in all its forms everywhere,"By 2030, build the resilience of the poor and ...",1.5,"['build', 'resilience', 'poor', 'vulnerable', ...",by build the resilience of the poor and those ...,"[['extreme', 'events'], ['poor', 'vulnerable']...",[[ 0.072906 0.157795 0.27599651 ... 0.7...,"[963, 964, 965, 966, 967, 968, 969, 970, 971, ...","[0.6873756433137852, 0.527061401761758, 0.5316...",60,"[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 5, 5,...","[17.3, 17.3, 17.3, 17.3, 17.3, 17.3, 17.3, 17....","['Birth,Registered events Year 2016, Registrat...","['Miscellaneous', 'Miscellaneous', 'Miscellane...","['Registration of Birth and Death', 'Registrat..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,16,Strengthen the means of implementation and rev...,"Enhance cooperation on and access to science, ...",16.3,"['enhance', 'cooperation', 'access', '

In [20]:
sdg_threshold = pd.read_csv(absolute_path_sem_threshold, dtype={'Goal No.': str, 'Target_id': str, 'Indicator_id':str}, sep=";", converters={'top_n_attr': pd.eval, 'top_n_table_id': pd.eval, 'top_n_chapter_id': pd.eval,'top_n_similarities': pd.eval})
sdg_threshold

,Goal No.,Goal,Targets,Target_id,new_targets,new_targets_str,keywords,vec,top_n_index,top_n_count,top_n_attr,top_n_chapter_id,top_n_table_id,top_n_description,top_n_chapter_name,top_n_table_name
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1.1,"['eradicate', 'extreme', 'poverty', 'people', ...",by eradicate extreme poverty for all people ev...,"[['poverty', 'people'], ['measured', 'people']...",[[-0.27236 0.229701 0.238934 ... 0.3...,"[132, 133, 376, 710, 711, 869, 870, 871, 875, ...",136,"[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[3, 3, 5, 11, 11, 14, 14, 14, 14, 3, 3, 3, 1, ...","[3.1, 3.1, 5.4, 11.6, 11.6, 14.3, 14.3, 14.3, ...","['Working,Raingauge Stations, No.of Rainguage ...","['Rainfall', 'Rainfall', 'Animal Husbandry', '...",['No.of Rainguage Raingauge Stations Annual No...
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",1.2,"['reduce', 'least', 'half', 'men', 'woman', 'c...",by reduce at least by half the proportion of m...,"[['reduce', 'least'], ['according', 'national'...",[[-0.070615 0.367065 0.00489179 ... -0.2...,"[594, 869, 511, 872, 873, 874, 875, 956, 833, ...",349,"[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[10, 14, 9, 14, 14, 14, 14, 17, 13, 13, 13, 13...","[10.11, 14.3, 9.2, 14.3, 14.3, 14.3, 14.3, 17....","['Men,No. of Lecturers, No.of P U Colleges Stu...","['Education', 'Rural Development & Panchayat R...",['No.of P U Colleges Students and Lecturers Ye...
2,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,1.3,"['implement', 'nationally', 'appropriate', 'so...",implement nationally appropriate social protec...,"[['appropriate', 'social'], ['protection', 'sy...",[[-0.60930499 -0.269645 -0.01956 ... 0.1...,"[198, 659, 660, 661, 662, 663, 664, 665, 666, ...",317,"[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[4, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 16...","[4.3, 11.3, 11.3, 11.3, 11.3, 11.3, 11.3, 11.3...","['Total Pulses including Tur Bengalgram,Pulses...","['Agriculture, Horticulture and Sericulture', ...","['Area under principal crops Year 2015-16', 'A..."
3,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",1.4,"['ensure', 'men', 'woman', 'particular', 'poor...",by ensure that all men and women in particular...,"[['well'], ['particular', 'poor'], ['financial...",[[-0.13508999 0.35907 0.1453 ... -0.4...,"[169, 170, 171, 172, 173, 174, 882, 883, 903, ...",491,"[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[4, 4, 4, 4, 4, 4, 14, 14, 15, 13, 13, 13, 13,...","[4.2, 4.2, 4.2, 4.2, 4.2, 4.2, 14.5, 14.5, 15....","['Nos,Wells,2015-16 Net area irrigated Hectare...","['Agriculture, Horticulture and Sericulture', ...",['Gross and Net area Irrigated under Different...
4,1,End poverty in all its forms everywhere,"By 2030, build the resilience of the poor and ...",1.5,"['build', 'resilience', 'poor', 'vulnerable', ...",by build the resilience of the poor and those ...,"[['extreme', 'events'], ['poor', 'vulnerable']...",[[ 0.072906 0.157795 0.27599651 ... 0.7...,"[963, 964, 965, 966, 967, 968, 969, 970, 971, ...",208,"[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 1...","[17.3, 17.3, 17.3, 17.3, 17.3, 17.3, 17.3, 17....","['Birth,Registered events Year 2016, Registrat...","['Miscellaneous', 'Miscellaneous', 'Miscellane...","['Registration of Birth and Death', 'Registrat..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,16,Strengthen the means of implementation and rev...,"Enhance cooperation on and access to science, ...",16.3,"['enhance', 'cooperation', 'access', 'science'...",enhance cooperation on and access to science t...,"[['global', 'technology'], ['among', 'existing...",[[ 0.023705 -0.0712105 0.05521001 ... -0.0...,"[198, 659, 660, 661, 662, 663, 664, 665, 666, ...",417,"[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","

In [21]:
result_sdg_new =sdg_string.copy()
result_sdg_sim_new =sdg_sim.copy()
result_sdg_threshold_new = sdg_threshold.copy()

In [22]:
result_sdg_new = result_sdg_new[['Goal No.', 'Target_id', 'top_n_attr']]

In [23]:
result_sdg_new.head()

,Goal No.,Target_id,top_n_attr
0,1,1.1,"[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35...."
1,1,1.2,"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839..."
2,1,1.3,"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672..."
3,1,1.4,"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672..."
4,1,1.5,"[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976..."


In [24]:
result_sdg_sim_new = result_sdg_sim_new[['Goal No.', 'Target_id', 'top_n_attr']]

In [25]:
result_sdg_sim_new.head()

,Goal No.,Target_id,top_n_attr
0,1,1.1,"[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877..."
1,1,1.2,"[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882..."
2,1,1.3,"[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671..."
3,1,1.4,"[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889..."
4,1,1.5,"[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976..."


In [26]:
result_sdg_threshold_new = result_sdg_threshold_new[['Goal No.', 'Target_id', 'top_n_attr']]

In [27]:
result_sdg_threshold_new.head()

,Goal No.,Target_id,top_n_attr
0,1,1.1,"[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877..."
1,1,1.2,"[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882..."
2,1,1.3,"[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671..."
3,1,1.4,"[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889..."
4,1,1.5,"[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976..."


In [28]:
result_sdg_new.rename(columns = {'top_n_attr':'top_n_attr_A'}, inplace=True)

In [29]:
result_sdg_new['top_n_attr_B'] = result_sdg_threshold_new.loc[:,'top_n_attr']
result_sdg_new['top_n_attr_C'] = result_sdg_sim_new.loc[:,'top_n_attr']

In [30]:
result_sdg_new

,Goal No.,Target_id,top_n_attr_A,top_n_attr_B,top_n_attr_C
0,1,1.1,"[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35....","[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877..."
1,1,1.2,"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882..."
2,1,1.3,"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671..."
3,1,1.4,"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889..."
4,1,1.5,"[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976..."
...,...,...,...,...,...
126,16,16.3,"[206.0, 523.0, 524.0, 525.0, 526.0, 527.0, 528...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671..."
127,16,16.4,"[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861..."
128,16,16.5,"[851.0, 852.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9...","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0..."
129,17,17.1,"[963.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 12...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851..."


In [31]:
# Apply the function to the merged dataframe
result_sdg_new['B_plus_C'] = result_sdg_new.apply(combine_lists_BC, axis=1)
result_sdg_new

,Goal No.,Target_id,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C
0,1,1.1,"[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35....","[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877..."
1,1,1.2,"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882..."
2,1,1.3,"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671..."
3,1,1.4,"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889..."
4,1,1.5,"[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976..."
...,...,...,...,...,...,...
126,16,16.3,"[206.0, 523.0, 524.0, 525.0, 526.0, 527.0, 528...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671..."
127,16,16.4,"[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861..."
128,16,16.5,"[851.0, 852.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9...","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0..."
129,17,17.1,"[963.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 12...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851..."


In [32]:
# Apply the function to the merged dataframe
result_sdg_new['A_and_BC'] = result_sdg_new.apply(find_intersection_ABC, axis=1)
result_sdg_new

,Goal No.,Target_id,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC
0,1,1.1,"[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35....","[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35...."
1,1,1.2,"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839..."
2,1,1.3,"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672..."
3,1,1.4,"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672..."
4,1,1.5,"[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976..."
...,...,...,...,...,...,...,...
126,16,16.3,"[206.0, 523.0, 524.0, 525.0, 526.0, 527.0, 528...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 523.0, 524.0, 525.0, 526.0, 527.0, 528..."
127,16,16.4,"[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845..."
128,16,16.5,"[851.0, 852.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9...","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[851.0, 852.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9..."
129,17,17.1,"[963.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 12...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851...","[963.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 12..."


In [33]:
# Apply the function to the merged dataframe
result_sdg_new['A'] = result_sdg_new.apply(find_difference_A, axis=1)
result_sdg_new

,Goal No.,Target_id,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC,A
0,1,1.1,"[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35....","[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35....",[]
1,1,1.2,"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35...."
2,1,1.3,"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...",[]
3,1,1.4,"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[27.0, 156.0, 157.0, 158.0, 159.0, 160.0, 161...."
4,1,1.5,"[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...",[]
...,...,...,...,...,...,...,...,...
126,16,16.3,"[206.0, 523.0, 524.0, 525.0, 526.0, 527.0, 528...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 523.0, 524.0, 525.0, 526.0, 527.0, 528...",[]
127,16,16.4,"[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...",[]
128,16,16.5,"[851.0, 852.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9...","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[851.0, 852.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9...","[384.0, 386.0]"
129,17,17.1,"[963.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 12...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851...","[963.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 12...","[384.0, 386.0, 575.0, 576.0, 577.0, 578.0, 579..."


In [34]:
# Apply the function to the merged dataframe
result_sdg_new['B_and_C'] = result_sdg_new.apply(find_intersection_BC, axis=1)
result_sdg_new

,Goal No.,Target_id,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC,A,B_and_C
0,1,1.1,"[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35....","[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35....",[],"[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877..."
1,1,1.2,"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35....","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882..."
2,1,1.3,"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...",[],"[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671..."
3,1,1.4,"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[27.0, 156.0, 157.0, 158.0, 159.0, 160.0, 161....","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889..."
4,1,1.5,"[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...",[],"[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976..."
...,...,...,...,...,...,...,...,...,...
126,16,16.3,"[206.0, 523.0, 524.0, 525.0, 526.0, 527.0, 528...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 523.0, 524.0, 525.0, 526.0, 527.0, 528...",[],"[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671..."
127,16,16.4,"[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...",[],"[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861..."
128,16,16.5,"[851.0, 852.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9...","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[851.0, 852.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9...","[384.0, 386.0]","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0..."
129,17,17.1,"[963.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 12...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851...","[963.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 12...","[384.0, 386.0, 575.0, 576.0, 577.0, 578.0, 579...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851..."


In [35]:
# Apply the function to the merged dataframe
result_sdg_new['B_minus_all'] = result_sdg_new.apply(find_difference_B, axis=1)
result_sdg_new

,Goal No.,Target_id,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC,A,B_and_C,B_minus_all
0,1,1.1,"[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35....","[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35....",[],"[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[439.0, 488.0, 489.0, 517.0, 656.0, 657.0, 842..."
1,1,1.2,"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35....","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[992.0, 993.0, 994.0, 995.0, 996.0, 997.0, 998..."
2,1,1.3,"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...",[],"[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[67.0, 68.0, 69.0, 91.0, 92.0, 93.0, 94.0, 95...."
3,1,1.4,"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[27.0, 156.0, 157.0, 158.0, 159.0, 160.0, 161....","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[493.0, 494.0, 495.0, 496.0, 866.0, 867.0, 868..."
4,1,1.5,"[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...",[],"[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[105.0, 106.0, 107.0, 108.0, 64.0, 65.0, 66.0,..."
...,...,...,...,...,...,...,...,...,...,...
126,16,16.3,"[206.0, 523.0, 524.0, 525.0, 526.0, 527.0, 528...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 523.0, 524.0, 525.0, 526.0, 527.0, 528...",[],"[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[593.0, 594.0, 595.0, 596.0, 597.0, 598.0, 599..."
127,16,16.4,"[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...",[],"[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29...."
128,16,16.5,"[851.0, 852.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9...","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[851.0, 852.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9...","[384.0, 386.0]","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[849.0, 850.0, 855.0, 856.0, 857.0, 858.0, 859..."
129,17,17.1,"[963.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 12...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851...","[963.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 12...","[384.0, 386.0, 575.0, 576.0, 577.0, 578.0, 579...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851...","[928.0, 931.0, 929.0, 930.0, 845.0, 846.0, 847..."


In [36]:
# Apply the function to the merged dataframe
result_sdg_new['C_minus_all'] = result_sdg_new.apply(find_difference_C, axis=1)
result_sdg_new

,Goal No.,Target_id,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC,A,B_and_C,B_minus_all,C_minus_all
0,1,1.1,"[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35....","[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35....",[],"[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[439.0, 488.0, 489.0, 517.0, 656.0, 657.0, 842...",[]
1,1,1.2,"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35....","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[992.0, 993.0, 994.0, 995.0, 996.0, 997.0, 998...",[]
2,1,1.3,"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...",[],"[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[67.0, 68.0, 69.0, 91.0, 92.0, 93.0, 94.0, 95....",[]
3,1,1.4,"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[27.0, 156.0, 157.0, 158.0, 159.0, 160.0, 161....","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[493.0, 494.0, 495.0, 496.0, 866.0, 867.0, 868...",[]
4,1,1.5,"[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...",[],"[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[105.0, 106.0, 107.0, 108.0, 64.0, 65.0, 66.0,...",[]
...,...,...,...,...,...,...,...,...,...,...,...
126,16,16.3,"[206.0, 523.0, 524.0, 525.0, 526.0, 527.0, 528...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 523.0, 524.0, 525.0, 526.0, 527.0, 528...",[],"[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[593.0, 594.0, 595.0, 596.0, 597.0, 598.0, 599...",[]
127,16,16.4,"[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...",[],"[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29....",[]
128,16,16.5,"[851.0, 852.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9...","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[851.0, 852.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9...","[384.0, 386.0]","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[849.0, 850.0, 855.0, 856.0, 857.0, 858.0, 859...",[]
129,17,17.1,"[963.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 12...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851...","[963.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 12...","[384.0, 386.0, 575.0, 576.0, 577.0, 578.0, 579...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851...","[928.0, 931.0, 929.0, 930.0, 845.0, 846.0, 847...",[]


In [37]:
result_sdg_new.columns

Index(['Goal No.', 'Target_id', 'top_n_attr_A', 'top_n_attr_B', 'top_n_attr_C',
       'B_plus_C', 'A_and_BC', 'A', 'B_and_C', 'B_minus_all', 'C_minus_all'],
      dtype='object')

In [38]:
# Apply the function to the merged dataframe
result_sdg_new['Rank'] = result_sdg_new.apply(final_rank, axis=1)
result_sdg_new

,Goal No.,Target_id,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC,A,B_and_C,B_minus_all,C_minus_all,Rank
0,1,1.1,"[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35....","[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35....",[],"[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[439.0, 488.0, 489.0, 517.0, 656.0, 657.0, 842...",[],"[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35...."
1,1,1.2,"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35....","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[992.0, 993.0, 994.0, 995.0, 996.0, 997.0, 998...",[],"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839..."
2,1,1.3,"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...",[],"[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[67.0, 68.0, 69.0, 91.0, 92.0, 93.0, 94.0, 95....",[],"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672..."
3,1,1.4,"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[27.0, 156.0, 157.0, 158.0, 159.0, 160.0, 161....","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[493.0, 494.0, 495.0, 496.0, 866.0, 867.0, 868...",[],"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672..."
4,1,1.5,"[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...",[],"[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[105.0, 106.0, 107.0, 108.0, 64.0, 65.0, 66.0,...",[],"[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976..."
...,...,...,...,...,...,...,...,...,...,...,...,...
126,16,16.3,"[206.0, 523.0, 524.0, 525.0, 526.0, 527.0, 528...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 523.0, 524.0, 525.0, 526.0, 527.0, 528...",[],"[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[593.0, 594.0, 595.0, 596.0, 597.0, 598.0, 599...",[],"[206.0, 523.0, 524.0, 525.0, 526.0, 527.0, 528..."
127,16,16.4,"[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...",[],"[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29....",[],"[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845..."
128,16,16.5,"[851.0, 852.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9...","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[851.0, 852.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9...","[384.0, 386.0]","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[849.0, 850.0, 855.0, 856.0, 857.0, 858.0, 859...",[],"[851.0, 852.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9..."
129,17,17.1,"[963.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 12...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851...","[963.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 12.

In [39]:
final_df = result_sdg_new.copy()

In [40]:
final_df['Attr_count'] = final_df['Rank'].apply(lambda x: len(x))

In [41]:
final_df

,Goal No.,Target_id,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC,A,B_and_C,B_minus_all,C_minus_all,Rank,Attr_count
0,1,1.1,"[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35....","[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35....",[],"[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[439.0, 488.0, 489.0, 517.0, 656.0, 657.0, 842...",[],"[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35....",136
1,1,1.2,"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35....","[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[992.0, 993.0, 994.0, 995.0, 996.0, 997.0, 998...",[],"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...",367
2,1,1.3,"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...",[],"[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[67.0, 68.0, 69.0, 91.0, 92.0, 93.0, 94.0, 95....",[],"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...",317
3,1,1.4,"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[27.0, 156.0, 157.0, 158.0, 159.0, 160.0, 161....","[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[493.0, 494.0, 495.0, 496.0, 866.0, 867.0, 868...",[],"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...",516
4,1,1.5,"[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...",[],"[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[105.0, 106.0, 107.0, 108.0, 64.0, 65.0, 66.0,...",[],"[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...",208
...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,16,16.3,"[206.0, 523.0, 524.0, 525.0, 526.0, 527.0, 528...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[206.0, 523.0, 524.0, 525.0, 526.0, 527.0, 528...",[],"[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[593.0, 594.0, 595.0, 596.0, 597.0, 598.0, 599...",[],"[206.0, 523.0, 524.0, 525.0, 526.0, 527.0, 528...",417
127,16,16.4,"[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...",[],"[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29....",[],"[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...",396
128,16,16.5,"[851.0, 852.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9...","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[851.0, 852.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9...","[384.0, 386.0]","[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0...","[849.0, 850.0, 855.0, 856.0, 857.0, 858.0, 859...",[],"[851.0, 852.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9...",390
129,17,17.1,"[963.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 12...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 851...","

In [42]:
final_df.to_csv(absolute_path_rank, sep=';', index=False)

In [43]:
all_goals = process_result(final_df, sdg, data)

In [44]:
all_goals.to_csv(absolute_path_rank_unroll, sep=';', index=False)

In [45]:
all_goals

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,Targets,top_n_goal_id,top_n_target_id
0,28.0,1,General Information,1.6,Ration Shops And B.p.l. Ration Card Holders on...,"Urban,Ration shops",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
1,29.0,1,General Information,1.6,Ration Shops And B.p.l. Ration Card Holders on...,"Rural,Ration shops",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
2,30.0,1,General Information,1.6,Ration Shops And B.p.l. Ration Card Holders on...,"Total,Ration shops",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
3,31.0,1,General Information,1.6,Ration Shops And B.p.l. Ration Card Holders on...,"Urban,Anthyodaya,Bpl Card Holders",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
4,32.0,1,General Information,1.6,Ration Shops And B.p.l. Ration Card Holders on...,"Rural,Anthyodaya,Bpl Card Holders",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
...,...,...,...,...,...,...,...,...,...,...
40318,586.0,10,Education,10.9,Infrastructure Facilities in Primary schools Y...,"Ramp,Infrastructure Facilitiesin Primary Schools","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
40319,587.0,10,Education,10.9,Infrastructure Facilities in Primary schools Y...,"Compound,Infrastructure Facilitiesin Primary S...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
40320,588.0,10,Education,10.9,Infrastructure Facilities in Primary schools Y...,"Drinking Water,Infrastructure Facilitiesin Pri...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
40321,589.0,10,Education,10.9,Infrastructure Facilities in Primary schools Y...,"Library,Infrastructure Facilitiesin Primary Sc...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2


In [46]:
# Group by 'col1' and count the distinct values in 'col2'
chap_map = data_new.groupby('Chapter_id')['Attr_id'].nunique().to_dict()
chap_map

{'1': 55,
 '10': 114,
 '11': 75,
 '12': 117,
 '13': 27,
 '14': 29,
 '15': 42,
 '16': 21,
 '17': 24,
 '18': 192,
 '2': 77,
 '3': 16,
 '4': 195,
 '5': 43,
 '6': 53,
 '7': 17,
 '8': 28,
 '9': 42}

In [47]:
# Group by 'col1' and count the distinct values in 'col2'
table_map = data_new.groupby('Chapter_id')['Table_id'].nunique().to_dict()
table_map

{'1': 10,
 '10': 15,
 '11': 6,
 '12': 6,
 '13': 6,
 '14': 5,
 '15': 7,
 '16': 3,
 '17': 3,
 '18': 1,
 '2': 11,
 '3': 2,
 '4': 18,
 '5': 4,
 '6': 5,
 '7': 2,
 '8': 4,
 '9': 4}

In [48]:
attr_map = data_new.groupby('Table_id')['Attr_id'].nunique().to_dict()
attr_map

{'1.1': 5,
 '1.10': 6,
 '1.2': 4,
 '1.3': 6,
 '1.4': 6,
 '1.5': 4,
 '1.6': 6,
 '1.7': 6,
 '1.8': 6,
 '1.9': 6,
 '10.1': 9,
 '10.10': 10,
 '10.11': 11,
 '10.12': 8,
 '10.13': 12,
 '10.14': 8,
 '10.15': 8,
 '10.2': 10,
 '10.3': 4,
 '10.4': 4,
 '10.5': 4,
 '10.6': 5,
 '10.7': 5,
 '10.8': 6,
 '10.9': 10,
 '11.1': 12,
 '11.2': 7,
 '11.3': 10,
 '11.4': 6,
 '11.5': 32,
 '11.6': 8,
 '12.1': 24,
 '12.2': 24,
 '12.3': 24,
 '12.4': 6,
 '12.5': 18,
 '12.6': 21,
 '13.1': 5,
 '13.2': 4,
 '13.3': 3,
 '13.4': 4,
 '13.5': 8,
 '13.6': 3,
 '14.1': 6,
 '14.2': 4,
 '14.3': 7,
 '14.4': 6,
 '14.5': 6,
 '15.1': 7,
 '15.2': 6,
 '15.3': 9,
 '15.4': 8,
 '15.5': 4,
 '15.6': 3,
 '15.7': 5,
 '16.1': 7,
 '16.2': 6,
 '16.3': 8,
 '17.1': 4,
 '17.2': 7,
 '17.3': 13,
 '18.1': 192,
 '2.1': 6,
 '2.11': 9,
 '2.12': 11,
 '2.2': 6,
 '2.3': 6,
 '2.4': 9,
 '2.5': 6,
 '2.6': 6,
 '2.7': 6,
 '2.8': 6,
 '2.9': 6,
 '3.1': 5,
 '3.2': 11,
 '4.1': 15,
 '4.10': 9,
 '4.11': 9,
 '4.12': 18,
 '4.13': 9,
 '4.14': 3,
 '4.15': 5,
 '4.16': 4,

In [49]:
all_goals

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,Targets,top_n_goal_id,top_n_target_id
0,28.0,1,General Information,1.6,Ration Shops And B.p.l. Ration Card Holders on...,"Urban,Ration shops",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
1,29.0,1,General Information,1.6,Ration Shops And B.p.l. Ration Card Holders on...,"Rural,Ration shops",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
2,30.0,1,General Information,1.6,Ration Shops And B.p.l. Ration Card Holders on...,"Total,Ration shops",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
3,31.0,1,General Information,1.6,Ration Shops And B.p.l. Ration Card Holders on...,"Urban,Anthyodaya,Bpl Card Holders",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
4,32.0,1,General Information,1.6,Ration Shops And B.p.l. Ration Card Holders on...,"Rural,Anthyodaya,Bpl Card Holders",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
...,...,...,...,...,...,...,...,...,...,...
40318,586.0,10,Education,10.9,Infrastructure Facilities in Primary schools Y...,"Ramp,Infrastructure Facilitiesin Primary Schools","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
40319,587.0,10,Education,10.9,Infrastructure Facilities in Primary schools Y...,"Compound,Infrastructure Facilitiesin Primary S...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
40320,588.0,10,Education,10.9,Infrastructure Facilities in Primary schools Y...,"Drinking Water,Infrastructure Facilitiesin Pri...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
40321,589.0,10,Education,10.9,Infrastructure Facilities in Primary schools Y...,"Library,Infrastructure Facilitiesin Primary Sc...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2


In [50]:
filter_df = all_goals.copy()

In [51]:
# Initialize an empty dictionary to store the counts
tab_result = {}
table_id_rm = {}
# Iterate over unique values in col1
for val in filter_df['top_n_target_id'].unique():
    # Filter the dataframe by the current value of col1
    filtered_df = filter_df[filter_df['top_n_target_id'] == val]
    
    # Count the occurrences of each value in col2 and store in a dictionary
    table_counts = filtered_df['Table_id'].value_counts().to_dict()
    
    table_id_rm[val] = {key:value for key, value in table_counts.items() if value < (attr_map[key]/2)}
    
    table_dict_new = {key:value for key, value in table_counts.items() if key not in table_id_rm[val]}
    
    if(len(table_dict_new) <= 4):
        # Sort the dictionary based on values in descending order
        sorted_dict = dict(sorted(table_counts.items(), key=lambda x: x[1], reverse=True))
        # Keep only the top 2 elements
        top_6_dict = dict(list(sorted_dict.items())[:6])
        tab_result[val] = top_6_dict
        
    else:
        # Add the col2 counts dictionary to the result dictionary
        tab_result[val] = table_dict_new

print(tab_result)

{'1.1': {'2.4': 9, '2.11': 9, '1.6': 6, '11.4': 6, '2.8': 6, '1.7': 6, '1.8': 6, '14.3': 5, '3.1': 5}, '1.2': {'18.1': 192, '11.5': 18, '6.1': 12, '11.3': 10, '2.4': 9, '2.11': 9, '13.5': 8, '10.8': 6, '14.3': 6, '10.2': 6, '10.1': 6, '2.3': 6, '1.8': 6, '1.6': 6, '1.7': 6, '2.5': 6, '11.4': 6, '11.6': 5, '13.1': 5, '13.2': 4, '10.15': 4, '10.14': 4, '13.4': 4, '13.3': 3, '13.6': 3}, '1.3': {'11.5': 32, '12.1': 24, '12.2': 24, '12.3': 24, '12.6': 21, '4.12': 18, '12.5': 18, '4.6': 18, '11.1': 12, '11.3': 10, '4.4': 10, '4.11': 9, '4.10': 9, '4.7': 9, '4.13': 9, '11.6': 8, '10.13': 8, '11.2': 7, '14.3': 7, '17.2': 7, '2.2': 6, '2.6': 6, '2.8': 6, '12.4': 6, '11.4': 6}, '1.4': {'18.1': 192, '11.5': 32, '12.3': 24, '12.2': 24, '12.1': 24, '12.6': 21, '12.5': 18, '4.1': 15, '11.1': 12, '11.3': 10, '2.11': 9, '13.5': 8, '11.6': 8, '14.3': 7, '17.2': 7, '11.2': 7, '2.2': 6, '12.4': 6, '14.4': 6, '9.3': 6, '11.4': 6, '2.6': 6, '14.5': 6, '14.1': 6, '2.8': 6, '13.1': 5, '13.2': 4, '13.4': 4, '

In [52]:
len(tab_result)

131

In [53]:
table_id_rm

{'1.1': {'18.1': 40,
  '4.2': 6,
  '6.1': 3,
  '5.4': 3,
  '4.18': 3,
  '8.4': 2,
  '5.1': 2,
  '6.4': 2,
  '11.6': 2,
  '14.5': 2,
  '11.1': 2,
  '15.5': 1,
  '16.3': 1,
  '16.2': 1,
  '15.7': 1,
  '4.3': 1,
  '13.4': 1,
  '13.1': 1,
  '9.1': 1,
  '15.3': 1,
  '5.2': 1,
  '2.1': 1},
 '1.2': {'10.7': 2,
  '10.6': 2,
  '14.1': 2,
  '17.2': 2,
  '2.1': 2,
  '1.4': 2,
  '10.11': 2,
  '15.3': 2,
  '9.2': 1},
 '1.3': {'4.3': 1, '16.2': 1, '13.6': 1},
 '1.4': {'4.2': 6,
  '9.1': 6,
  '6.4': 2,
  '10.11': 2,
  '4.3': 1,
  '1.5': 1,
  '15.3': 1,
  '8.2': 1,
  '16.2': 1},
 '1.5': {'5.4': 2, '16.2': 2, '15.3': 1, '4.3': 1},
 '1.6': {'4.2': 6, '11.5': 6, '6.4': 2, '9.2': 1, '15.3': 1},
 '2.1': {'18.1': 48,
  '4.2': 6,
  '9.1': 6,
  '6.4': 4,
  '15.3': 3,
  '4.3': 3,
  '4.18': 3,
  '6.1': 3,
  '8.4': 2,
  '11.5': 2,
  '11.1': 2,
  '13.4': 1,
  '4.4': 1,
  '4.12': 1,
  '16.2': 1,
  '5.4': 1,
  '5.1': 1,
  '5.2': 1,
  '11.4': 1,
  '15.5': 1,
  '13.6': 1,
  '4.6': 1,
  '13.1': 1,
  '16.3': 1,
  '15.7

In [54]:
sum_ind=0
# Iterate over the outer dictionary
for key, inner_dict in table_id_rm.items():
    # Iterate over the inner dictionary
    for k, value in inner_dict.items():
        if k not in tab_result[key]:
            # Add the value to the result
            sum_ind += value

print(sum_ind)

3522


In [55]:
# List to store indices to remove
indices_to_remove = []
# Iterate over the rows of the DataFrame
for index, row in filter_df.iterrows():
    if row['top_n_target_id'] in table_id_rm:
        inner_dict = table_id_rm[row['top_n_target_id']]
        if row['Table_id'] in inner_dict:
            if row['Table_id'] not in tab_result[row['top_n_target_id']]:
                # Add index to remove list
                indices_to_remove.append(index)
            
indices_to_remove

[23,
 24,
 25,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 128,
 135,
 169,
 194,
 195,
 196,
 203,
 204,
 211,
 252,
 253,
 446,
 447,
 473,
 474,
 487,
 488,
 501,
 502,
 513,
 650,
 651,
 861,
 862,
 863,
 864,
 865,
 866,
 867,
 868,
 869,
 870,
 938,
 1131,
 1147,
 1154,
 1155,
 1156,
 1157,
 1158,
 1159,
 1160,
 1326,
 1348,
 1349,
 1368,
 1370,
 1396,
 1420,
 1562,
 1619,
 1620,
 1621,
 1622,
 1623,
 1624,
 1625,
 1626,
 1691,
 1692,
 1693,
 1694,
 1695,
 1696,
 1697,
 1703,
 1704,
 1705,
 1706,
 1707,
 1708,
 1725,
 1726,
 1727,
 1728,
 1729,
 1730,
 1731,
 1732,
 1733,
 1734,
 1735,
 1736,
 1737,
 1738,
 1739,
 1745,
 1746,
 1747,
 1753,
 1754,


In [56]:
len(indices_to_remove)

3522

In [57]:
# Remove rows using indices from indices_to_remove list
filter_df.drop(indices_to_remove, inplace=True)

filter_df

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,Targets,top_n_goal_id,top_n_target_id
0,28.0,1,General Information,1.6,Ration Shops And B.p.l. Ration Card Holders on...,"Urban,Ration shops",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
1,29.0,1,General Information,1.6,Ration Shops And B.p.l. Ration Card Holders on...,"Rural,Ration shops",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
2,30.0,1,General Information,1.6,Ration Shops And B.p.l. Ration Card Holders on...,"Total,Ration shops",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
3,31.0,1,General Information,1.6,Ration Shops And B.p.l. Ration Card Holders on...,"Urban,Anthyodaya,Bpl Card Holders",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
4,32.0,1,General Information,1.6,Ration Shops And B.p.l. Ration Card Holders on...,"Rural,Anthyodaya,Bpl Card Holders",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
...,...,...,...,...,...,...,...,...,...,...
40318,586.0,10,Education,10.9,Infrastructure Facilities in Primary schools Y...,"Ramp,Infrastructure Facilitiesin Primary Schools","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
40319,587.0,10,Education,10.9,Infrastructure Facilities in Primary schools Y...,"Compound,Infrastructure Facilitiesin Primary S...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
40320,588.0,10,Education,10.9,Infrastructure Facilities in Primary schools Y...,"Drinking Water,Infrastructure Facilitiesin Pri...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
40321,589.0,10,Education,10.9,Infrastructure Facilities in Primary schools Y...,"Library,Infrastructure Facilitiesin Primary Sc...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2


In [58]:
filter_df.to_csv(absolute_path_tab_filter, sep=';', index=False)

In [59]:
tab_count_att = filter_df['top_n_target_id'].value_counts().to_dict()

In [60]:
filter_df_copy = filter_df.copy()

In [61]:
def filter_dataframe(df, n, chap_list):
    result_df = df.copy() # Initialize an empty DataFrame to store the filtered rows
    
    # Loop over each unique 'Target_id'
    for target_id, group in df.groupby('top_n_target_id'):
        num_rows = len(group)  # Count the number of rows for this 'Target_id'
        
        # Check if the number of rows is greater than the threshold 'n'
        if num_rows > n:
            idx_list = []
            drop_list = []
            for chap in chap_list:
                # Find the indices corresponding to chapters "2" and "3" for this 'Target_id'
                idx_chapter = group[group['Chapter_id'] == chap].index
                idx_list.append(idx_chapter)
                
            for item in idx_list:
                if len(item)>1:
                    total_rows_to_drop = item[len(item)//2:]
                    drop_list.append(total_rows_to_drop)
            
            for i in drop_list:
                if len(i)>0:
                    result_df = result_df.drop(i)
    result_df.reset_index(inplace = True, drop = True)
    return result_df

In [62]:
chap_ll = ['2', '4', '10', '12', '18']
# Filter the DataFrame using the function
final_map = filter_dataframe(filter_df_copy, 350, chap_ll)

In [63]:
final_map.to_csv(absolute_path_final_map, sep=';', index=False)

In [64]:
chapter_df = filter_df.copy()

In [65]:
# Initialize an empty dictionary to store the counts
chap_result = {}
chap_id_rm = {}

# Iterate over unique values in col1
for val in chapter_df['top_n_target_id'].unique():
    # Filter the dataframe by the current value of col1
    filtered_df = chapter_df[chapter_df['top_n_target_id'] == val]
    chap_list = []
    for tab in filtered_df['Table_id'].unique():
        chap_filter = filtered_df[filtered_df['Table_id'] == tab]
        
        # Count the occurrences of each value in col2 and store in a dictionary
        chap_counts = chap_filter['Chapter_id'].unique().tolist()
        
        chap_list.extend(chap_counts)
    chap_dict = pd.Series(chap_list).value_counts().to_dict()
    chap_id_rm[val] = {key:value for key, value in chap_dict.items() if value < (table_map[key]/2)}
    chap_dict_new = {key:value for key, value in chap_dict.items() if key not in chap_id_rm[val]}
    if(len(chap_dict_new) <= 1):
        # Sort the dictionary based on values in descending order
        sorted_dict = dict(sorted(chap_dict.items(), key=lambda x: x[1], reverse=True))
        # Keep only the top 2 elements
        top_4_dict = dict(list(sorted_dict.items())[:4])
        chap_result[val] = top_4_dict
 
    else:
        # Add the col2 counts dictionary to the result dictionary
        chap_result[val] = chap_dict_new

print(chap_result)

{'1.1': {'1': 3, '2': 3, '3': 1, '14': 1}, '1.2': {'13': 6, '11': 4, '18': 1}, '1.3': {'11': 6, '12': 6}, '1.4': {'11': 6, '13': 6, '12': 6, '14': 5, '18': 1}, '1.5': {'13': 3, '17': 2}, '1.6': {'13': 6, '14': 5}, '2.1': {'14': 3, '13': 3, '3': 1}, '2.2': {'13': 6, '11': 3, '18': 1}, '2.3': {'1': 10, '11': 6, '8': 4, '18': 1}, '2.4': {'8': 4, '18': 1}, '2.5': {'1': 10, '5': 4, '11': 3, '13': 3, '8': 2, '7': 2, '18': 1}, '2.6': {'4': 10, '11': 6, '13': 6, '14': 5, '8': 4, '7': 2, '5': 2, '18': 1}, '2.7': {'13': 6, '14': 5, '8': 4, '18': 1}, '2.8': {'1': 10, '11': 3, '13': 3, '18': 1}, '3.1': {'13': 6, '17': 2}, '3.2': {'13': 6, '10': 4, '2': 3, '17': 1}, '3.3': {'11': 6, '3': 2}, '3.4': {'11': 6, '13': 6}, '3.5': {'9': 4, '11': 3}, '3.6': {'10': 15, '1': 10, '11': 6, '18': 1}, '3.7': {'4': 7, '11': 6, '9': 1, '10': 1}, '3.8': {'9': 4, '17': 2}, '3.9': {'11': 6, '13': 6, '14': 5, '18': 1, '7': 1}, '3.10': {'1': 10, '11': 6, '13': 6, '14': 5, '12': 4, '18': 1}, '3.11': {'11': 6, '13': 6, 

In [66]:
chap_id_rm

{'1.1': {'1': 3, '2': 3, '14': 1, '11': 1},
 '1.2': {'10': 5, '2': 4, '1': 3, '14': 1, '6': 1},
 '1.3': {'4': 7, '2': 3, '10': 1, '17': 1, '14': 1},
 '1.4': {'2': 4, '4': 1, '9': 1, '17': 1},
 '1.5': {'4': 8, '2': 4, '1': 3, '11': 2, '16': 1, '14': 1},
 '1.6': {'10': 5, '1': 3, '2': 3, '12': 1, '11': 1},
 '2.1': {'11': 2, '10': 2, '9': 1, '1': 1},
 '2.2': {'2': 4, '1': 3, '15': 2, '10': 1, '4': 1, '14': 1},
 '2.3': {'6': 2, '13': 2},
 '2.4': {'4': 7,
  '2': 3,
  '13': 2,
  '10': 2,
  '11': 2,
  '5': 1,
  '9': 1,
  '16': 1,
  '17': 1,
  '15': 1},
 '2.5': {'4': 8, '10': 2, '9': 1, '14': 1},
 '2.6': {'15': 2, '10': 2, '2': 1, '9': 1},
 '2.7': {'11': 1},
 '2.8': {'9': 1, '15': 1},
 '3.1': {'2': 3, '10': 3, '16': 1, '11': 1},
 '3.2': {'10': 4, '2': 3, '17': 1, '11': 1, '12': 1},
 '3.3': {'1': 3, '14': 2, '9': 1, '15': 1},
 '3.4': {'2': 4, '14': 2, '10': 2, '15': 1},
 '3.5': {'17': 1, '5': 1, '14': 1},
 '3.6': {'13': 2, '14': 1, '9': 1},
 '3.7': {'4': 7, '9': 1, '10': 1, '13': 1},
 '3.8': {'

In [67]:
missing = []

# Iterate through the data dictionary
for key, value in chap_result.items():
    # Check if the sub-dictionary is empty
    if not value:
        # Add the key to the missing list
        missing.append(key)
missing

[]

In [68]:
# List to store indices to remove
chap_indices_to_remove = []
# Iterate over the rows of the DataFrame
for index, row in chapter_df.iterrows():
    if row['top_n_target_id'] in table_id_rm:
        inner_dict = chap_id_rm[row['top_n_target_id']]
        if row['Chapter_id'] in inner_dict:
            if row['top_n_target_id'] not in chap_result or row['Chapter_id'] not in chap_result[row['top_n_target_id']]:
                # Add index to remove list
                chap_indices_to_remove.append(index)

chap_indices_to_remove

[129,
 130,
 131,
 132,
 133,
 134,
 136,
 137,
 138,
 139,
 140,
 141,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 205,
 206,
 207,
 208,
 209,
 210,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 254,
 255,
 256,
 257,
 258,
 259,
 434,
 435,
 436,
 437,
 438,
 439,
 440,
 441,
 442,
 443,
 444,
 445,
 448,
 449,
 450,
 451,
 475,
 476,
 477,
 478,
 479,
 480,
 481,
 482,
 483,
 484,
 485,
 486,
 489,
 490,
 491,
 492,
 514,
 515,
 516,
 517,
 518,
 519,
 520,
 521,
 652,
 653,
 654,
 655,
 656,
 657,
 658,
 659,
 660,
 661,
 662,
 663,
 664,
 665,
 672,
 673,
 674,
 675,
 676,
 677,
 678,
 679,
 680,
 681,
 682,
 683,
 684,
 685,
 686,
 687,
 688,
 689,
 738,
 739,
 740,
 741,
 742,
 743,
 744,
 745,
 746,
 747,
 748,
 749,
 750,
 751,
 752,
 753,
 754,
 755,
 756,
 757,
 758,
 759,
 760,
 761,
 762,
 763,
 764,
 765,
 766

In [69]:
len(chap_indices_to_remove)

7901

In [70]:
# Remove rows using indices from indices_to_remove list
chapter_df.drop(chap_indices_to_remove, inplace=True)

chapter_df

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,Targets,top_n_goal_id,top_n_target_id
0,28.0,1,General Information,1.6,Ration Shops And B.p.l. Ration Card Holders on...,"Urban,Ration shops",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
1,29.0,1,General Information,1.6,Ration Shops And B.p.l. Ration Card Holders on...,"Rural,Ration shops",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
2,30.0,1,General Information,1.6,Ration Shops And B.p.l. Ration Card Holders on...,"Total,Ration shops",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
3,31.0,1,General Information,1.6,Ration Shops And B.p.l. Ration Card Holders on...,"Urban,Anthyodaya,Bpl Card Holders",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
4,32.0,1,General Information,1.6,Ration Shops And B.p.l. Ration Card Holders on...,"Rural,Anthyodaya,Bpl Card Holders",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
...,...,...,...,...,...,...,...,...,...,...
40277,957.0,16,Energy,16.3,Sectorwise Electricity Consumption Year 2016-17,Income from electricity consumption Rs. in lakh,"Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
40282,945.0,16,Energy,16.2,Sectorwise Electricity Consumers on In Nos,Industrial,"Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
40283,947.0,16,Energy,16.2,Sectorwise Electricity Consumers on In Nos,I.P.Sets,"Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
40284,948.0,16,Energy,16.2,Sectorwise Electricity Consumers on In Nos,Others including Street Lights,"Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2


In [71]:
chapter_df['top_n_target_id'].nunique()

131

In [72]:
chap_count_att = chapter_df['top_n_target_id'].value_counts().to_dict()

In [73]:
chap_count_att

{'10.3': 542,
 '2.6': 502,
 '11.8': 469,
 '1.4': 440,
 '3.6': 436,
 '3.10': 426,
 '15.7': 425,
 '14.6': 420,
 '4.8': 414,
 '15.9': 413,
 '13.3': 405,
 '4.9': 403,
 '4.4': 399,
 '12.5': 398,
 '11.4': 384,
 '4.3': 377,
 '11.1': 376,
 '16.5': 374,
 '5.7': 371,
 '15.6': 367,
 '2.5': 359,
 '16.3': 356,
 '6.2': 354,
 '2.3': 350,
 '5.8': 345,
 '13.1': 344,
 '9.4': 343,
 '10.2': 341,
 '3.9': 335,
 '12.4': 335,
 '4.5': 333,
 '14.9': 331,
 '16.4': 329,
 '8.10': 326,
 '3.11': 323,
 '14.4': 323,
 '14.1': 323,
 '11.9': 323,
 '11.6': 320,
 '15.4': 315,
 '6.1': 315,
 '15.3': 308,
 '12.7': 303,
 '9.8': 301,
 '5.6': 298,
 '2.8': 297,
 '15.8': 293,
 '14.2': 287,
 '2.7': 276,
 '14.7': 273,
 '8.9': 271,
 '8.3': 267,
 '4.2': 266,
 '7.2': 266,
 '8.11': 263,
 '1.2': 258,
 '17.1': 257,
 '8.6': 257,
 '14.8': 256,
 '4.10': 248,
 '14.10': 246,
 '2.2': 243,
 '14.11': 239,
 '10.4': 231,
 '5.2': 225,
 '12.9': 223,
 '2.4': 220,
 '6.4': 217,
 '14.3': 208,
 '16.1': 206,
 '4.1': 204,
 '9.3': 196,
 '1.3': 192,
 '5.4': 1

In [74]:
tab_count_att

{'10.3': 578,
 '11.8': 553,
 '2.6': 541,
 '3.10': 533,
 '11.4': 510,
 '4.4': 505,
 '15.7': 503,
 '1.4': 495,
 '2.5': 481,
 '4.9': 473,
 '3.6': 458,
 '17.1': 456,
 '4.8': 445,
 '14.6': 444,
 '15.9': 440,
 '14.1': 438,
 '12.5': 430,
 '5.7': 429,
 '13.3': 428,
 '13.1': 426,
 '4.3': 419,
 '15.3': 418,
 '14.4': 417,
 '6.2': 416,
 '15.6': 411,
 '9.4': 409,
 '16.3': 404,
 '3.9': 403,
 '11.1': 395,
 '4.5': 388,
 '2.3': 387,
 '2.4': 385,
 '16.5': 382,
 '6.1': 382,
 '16.4': 380,
 '11.9': 378,
 '12.4': 377,
 '10.2': 375,
 '5.6': 374,
 '14.9': 358,
 '14.2': 357,
 '8.3': 355,
 '5.8': 353,
 '1.2': 350,
 '15.4': 349,
 '8.10': 343,
 '3.11': 343,
 '5.2': 341,
 '5.4': 338,
 '9.8': 333,
 '4.1': 330,
 '11.6': 327,
 '2.2': 325,
 '14.7': 322,
 '6.8': 322,
 '15.8': 315,
 '1.3': 314,
 '8.9': 314,
 '2.8': 310,
 '7.2': 306,
 '12.7': 303,
 '4.10': 299,
 '14.10': 297,
 '14.8': 296,
 '4.2': 296,
 '8.6': 288,
 '2.7': 286,
 '14.3': 286,
 '11.5': 283,
 '8.11': 278,
 '10.4': 276,
 '14.11': 274,
 '7.1': 272,
 '6.4': 26

In [75]:
rank_count_att = all_goals['top_n_target_id'].value_counts().to_dict()

In [78]:
rank_filter_count_att = final_map['top_n_target_id'].value_counts().to_dict()

In [79]:
count_att_list = [(key, value) for key, value in chap_count_att.items()]

count_att_df = pd.DataFrame(count_att_list, columns=['Target_id', 'Chap_count'])
count_att_df

,Target_id,Chap_count
0,10.3,542
1,2.6,502
2,11.8,469
3,1.4,440
4,3.6,436
...,...,...
126,2.1,35
127,6.3,33
128,15.2,32
129,1.5,27


In [80]:
count_att_df['Tab_count'] = count_att_df['Target_id'].map(tab_count_att)

In [81]:
count_att_df['Filter_count'] = count_att_df['Target_id'].map(rank_filter_count_att)

In [82]:
count_att_df['Rank_count'] = count_att_df['Target_id'].map(rank_count_att)

In [83]:
count_att_df

,Target_id,Chap_count,Tab_count,Filter_count,Rank_count
0,10.3,542,578,342,606
1,2.6,502,541,377,597
2,11.8,469,553,327,575
3,1.4,440,495,318,516
4,3.6,436,458,305,467
...,...,...,...,...,...
126,2.1,35,74,74,171
127,6.3,33,41,41,54
128,15.2,32,102,102,117
129,1.5,27,202,202,208


In [84]:
chapter_df.to_csv(absolute_path_chap_filter, sep=';', index=False)

In [85]:
count_att_df.to_csv(absolute_path_count, sep=';', index=False)